## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras
import itertools

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
"""Code Here
設定超參數
"""

LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [43]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
filepaths = ""
from keras.callbacks import ModelCheckpoint

# testing different parameter
save_best_onlys = [True, False]
save_weights_onlys = [True, False]
for save_best_only, save_weights_only in itertools.product(save_best_onlys ,save_weights_onlys):
    filepaths = "./tmp_%s-%s.h5"% (str(save_best_only), str(save_weights_only))
    print (filepaths)
    
    
    model_ckpt = ModelCheckpoint(filepath=filepaths, 
                                 monitor="val_loss", 
                                 save_best_only=save_best_only,
                                 save_weights_only= save_weights_only)



    results = {}
    """Code Here
    撰寫你的訓練流程並將結果用 dictionary 紀錄
    """

    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[model_ckpt]
             )

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    exp_name_tag = filepaths 
    results[exp_name_tag] = {'train-loss': train_loss,
                                    'valid-loss': valid_loss,
                                    'train-acc': train_acc,
                                    'valid-acc': valid_acc}


./tmp_True-True.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_16 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_17 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_18 (Batc (None, 128)               51

50000/50000 [==============================] - 6s 120us/step - loss: 0.5002 - acc: 0.8603 - val_loss: 1.6624 - val_acc: 0.4919
Epoch 50/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.4838 - acc: 0.8667 - val_loss: 1.7018 - val_acc: 0.4890
./tmp_True-False.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_19 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_20 (Batc (None, 256)              

50000/50000 [==============================] - 6s 121us/step - loss: 0.5417 - acc: 0.8441 - val_loss: 1.6363 - val_acc: 0.4970
Epoch 48/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.5266 - acc: 0.8489 - val_loss: 1.6811 - val_acc: 0.4874
Epoch 49/50
50000/50000 [==============================] - 6s 118us/step - loss: 0.5111 - acc: 0.8554 - val_loss: 1.6873 - val_acc: 0.4888
Epoch 50/50
50000/50000 [==============================] - 6s 120us/step - loss: 0.4954 - acc: 0.8602 - val_loss: 1.6894 - val_acc: 0.4886
./tmp_False-True.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_22 (Batc (None, 512)

50000/50000 [==============================] - 6s 118us/step - loss: 0.5604 - acc: 0.8377 - val_loss: 1.6272 - val_acc: 0.4918
Epoch 46/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.5414 - acc: 0.8445 - val_loss: 1.6480 - val_acc: 0.4891
Epoch 47/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.5252 - acc: 0.8514 - val_loss: 1.6372 - val_acc: 0.4965
Epoch 48/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.5054 - acc: 0.8587 - val_loss: 1.6538 - val_acc: 0.4936
Epoch 49/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.4885 - acc: 0.8659 - val_loss: 1.6601 - val_acc: 0.4946
Epoch 50/50
50000/50000 [==============================] - 6s 118us/step - loss: 0.4752 - acc: 0.8701 - val_loss: 1.6834 - val_acc: 0.4866
./tmp_False-False.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)     

50000/50000 [==============================] - 6s 117us/step - loss: 0.6212 - acc: 0.8120 - val_loss: 1.6168 - val_acc: 0.4848
Epoch 44/50
50000/50000 [==============================] - 6s 125us/step - loss: 0.6041 - acc: 0.8186 - val_loss: 1.6063 - val_acc: 0.4936
Epoch 45/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.5862 - acc: 0.8247 - val_loss: 1.6174 - val_acc: 0.4898
Epoch 46/50
50000/50000 [==============================] - 6s 120us/step - loss: 0.5680 - acc: 0.8325 - val_loss: 1.6113 - val_acc: 0.4906
Epoch 47/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.5520 - acc: 0.8392 - val_loss: 1.6208 - val_acc: 0.4916
Epoch 48/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.5352 - acc: 0.8472 - val_loss: 1.6406 - val_acc: 0.4885
Epoch 49/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.5178 - acc: 0.8531 - val_loss: 1.6564 - val_acc: 0.4909
Epoch 50/50
50000/50000 [==============

In [86]:
import itertools
save_best_onlys = [True, False]
save_weights_onlys = [True, False]
filepaths1 = []
filepaths2 = []

#filepaths2 saved only weights, need to rebuild model 

for save_best_only in save_best_onlys:
    filepaths1.append("./tmp_%s-%s.h5"% (str(save_best_only), "False"))
    filepaths2.append("./tmp_%s-%s.h5"% (str(save_best_only), "True"))
#print (filepaths1)
#print (filepaths2)
"""Code Here
將結果繪出
"""
for filepaths1 in filepaths1:
    # Load back true
    print(filepaths1)
    model = keras.models.load_model(filepaths1)
    loss_loadback, acc_loadback = model.evaluate(x_test, y_test)
    print(loss_loadback, acc_loadback,"\n")
    
for filepaths2 in filepaths2:
#filepaths2 saved only weights, need to rebuild model 
    print(filepaths2)
    model = build_mlp(input_shape=x_train.shape[1:])
    model.load_weights(filepaths2)
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)
    loss_loadback, acc_loadback = model.evaluate(x_test, y_test)
    print(loss_loadback, acc_loadback,"\n")



./tmp_True-False.h5
10000/10000 [==============================] - 3s 315us/step
1.4553574750900269 0.497 

./tmp_False-False.h5
10000/10000 [==============================] - 3s 315us/step
1.6621826717376709 0.486 

./tmp_True-True.h5
10000/10000 [==============================] - 5s 517us/step
1.4472202531814575 0.4932 

./tmp_False-True.h5
10000/10000 [==============================] - 5s 517us/step
1.6833523672103883 0.4866 

